In [9]:
import cv2
import numpy as np

image = cv2.imread('imgnotmalization.jpg')
image = cv2.resize(cl1, (640, 640))
cv2.imshow('CLAHE', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
import cv2
import numpy as np

image = cv2.imread('imgnotmalization.jpg')
denoised = cv2.medianBlur(image, 5)
gray_image = cv2.cvtColor(denoised, cv2.COLOR_BGR2GRAY)

In [26]:
import telebot
import io
import tempfile
import torch
from ultralytics import YOLO
import cv2
import numpy as np

# Вставьте сюда ваш токен
TOKEN = '7178794179:AAFrz0ImoWU2FGDDTKRx3lIS2dd9ZQY89aU'
bot = telebot.TeleBot(TOKEN)

# Обработчик для команды /start
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Добрый день! Я бот который находит дефекты на сварных швах. Отправь мне изображение, а я отмечу на нем дефекты!")


@bot.message_handler(content_types=['photo'])
def handle_image(message):
    # Получаем файл изображения
    file_info = bot.get_file(message.photo[-1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    # Чтение изображения из памяти
    image_np = np.frombuffer(downloaded_file, np.uint8)
    image = cv2.imdecode(image_np, cv2.IMREAD_COLOR)
    model = YOLO('runs/detect/train2/weights/best.pt')
    test_images = [image]
    results = model.predict(source=test_images, device='cuda' if torch.cuda.is_available() else 'cpu', imgsz=640)

    # Просмотр результатов
    for i, result in enumerate(results):
        # Получение изображения с предсказанными аннотациями
        annotated_frame = result.plot()  # Метод plot() возвращает изображение с аннотациями
        output_path = f"annotated_image_{i+1}.jpg"
        cv2.imwrite(output_path, annotated_frame)
        defects = result.boxes
        if not defects:
            bot.send_message(message.chat.id, "Дефекты не обнаружены")
        else:
            with open(output_path, 'rb') as img_file:
                bot.send_photo(message.chat.id, img_file)



# Запуск бота
bot.polling()



0: 384x640 (no detections), 149.7ms
Speed: 2.0ms preprocess, 149.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1  , 142.6ms
Speed: 2.0ms preprocess, 142.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
